<a href="https://colab.research.google.com/github/stefanmzeidler/HI744_Programming_Assignment_1/blob/main/Zeidler_Programming_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installs, downloads, and imports

In [1]:
!pip install gensim
!pip install nltk

In [43]:
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd


# Util Functions

In [3]:
#Provided by Professor He
import nltk
import json
import os
from collections import defaultdict
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import json
import sys

def read_txt_files_from_directory(directory_path):
    file_contents = {}
    try:
        for filename in os.listdir(directory_path):
            if filename.endswith('.txt'):
                file_path = os.path.join(directory_path, filename)
                try:
                    with open(file_path, 'r', encoding='utf-8') as file:
                        file_contents[filename] = file.read()
                except Exception as e:
                    print(f"An error occurred while reading {filename}: {e}")
    except Exception as e:
        print(f"An error occurred while accessing the directory: {e}")
        return {}
    return file_contents

def load_from_json(filename):
    try:
        with open(filename, 'r') as json_file:
            data = json.load(json_file)
        return data
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return None

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(text)

def remove_stop_words(tokens):
    stop_words = set(stopwords.words('english'))
    filtered_sentence = []
    for token in tokens:
        if token not in stop_words:
            filtered_sentence.append(token)
    return filtered_sentence

def stemming(tokens):
    stemmer = PorterStemmer()
    return [stemmer.stem(token) for token in tokens]

def pre_process(text):
    text_lower = text.lower()
    tokens_no_punctuation = remove_punctuation(text_lower)
    filtered_tokens = remove_stop_words(tokens_no_punctuation)
    stemmed_tokens = stemming(filtered_tokens)
    return stemmed_tokens

In [51]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile

def load_dataset(fname,nrows):
  data = safe_read_csv(fname,nrows)
  data['tokens'] = data['patient'].apply(lambda text: pre_process(text))
  return data

def safe_read_csv(fname,nrows):
  try:
    filepath = os.path.join(PROJ_DIRECTORY,fname)
    return pd.read_csv(filepath_or_buffer=filepath,nrows=1000)
  except Exception as e:
    print(f"An error occurred while reading the file: {e}")
    return None

#Adapted from https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html#sphx-glr-auto-examples-tutorials-run-doc2vec-lee-py
def read_corpus(data, tokens_only = False):
    documents = data['tokens']
    for i, tokens in documents.items():
        if tokens_only:
            yield tokens
        else:
            # For training data, add tags
            yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

def train_model(data_file,nrows = 1000):
  model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=1, epochs=40)
  data = load_dataset(data_file,nrows)
  print("Data loaded")
  train_corpus = list(read_corpus(data))
  test_corpus = list(read_corpus(data, tokens_only=True))
  model.build_vocab(train_corpus)
  print("Vocab built")
  print("Starting training")
  model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)
  fname = get_tmpfile(os.path.join(PROJ_DIRECTORY,"my_doc2vec_model"))
  model.save(fname)
  print("Model saved")
  return model

def calc_vectors(model, data):
  data['vector'] = data['tokens'].apply(lambda tokens: model.infer_vector(tokens))


def load_model(fname):
  return Doc2Vec.load(fname)






In [52]:
PROJ_DIRECTORY ="/content/drive/MyDrive/HI744_Programming_Assignment_1"
model = train_model("PMC-Patients.csv")
# PROJ_DIRECTORY = os.getcwd()
# print(data.head())
# train_corpus = list(read_corpus(data))
# test_corpus = list(read_corpus(data, tokens_only=True))
# print(train_corpus[:3])
# model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=1, epochs=40)
# model.build_vocab(train_corpus)
# model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)
# print(data['tokens'][1])
# vector = model.infer_vector(data['tokens'][1])
# print(vector)





Data loaded
Vocab built
Starting training
model saved
